<a href="https://colab.research.google.com/github/Maks6666/ML/blob/main/%D0%94%D0%97_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [136]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [137]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module2/data/ship_passengers.csv",
                 index_col="Unnamed: 0")

In [138]:
df.head()

,pclass,sex,age,sibsp,parch,fare,embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [139]:
df = df[['pclass', 'age', 'embarked', 'fare']]
df.head()

,pclass,age,embarked,fare
0,3,22.0,S,7.2500
1,1,38.0,C,71.2833
2,3,26.0,S,7.9250
3,1,35.0,S,53.1000
4,3,35.0,S,8.0500


# Завдання 1
Очистіть дані від викидів(лише `fare`), розділіть на тренувальну та тестову чатини

In [140]:
df.shape

(891, 4)

In [141]:
df.isnull().sum()

pclass        0
age         177
embarked      2
fare          0
dtype: int64

In [142]:
df.describe()

,pclass,age,fare
count,891.000000,714.000000,891.000000
mean,2.308642,29.699118,32.204208
std,0.836071,14.526497,49.693429
min,1.000000,0.420000,0.000000
25%,2.000000,20.125000,7.910400
50%,3.000000,28.000000,14.454200
75%,3.000000,38.000000,31.000000
max,3.000000,80.000000,512.329200


In [143]:
def outliers(data, column):
  column = data[column]
  q1 = column.quantile(q=0.25)
  q3 = column.quantile(q=0.75)
  iqr = q3 - q1

  lower = q1 - 1.5 * iqr
  upper = q3 + 1.5 * iqr


  mask = (column >= lower) & (column <= upper)

  return data[mask]

data = outliers(df, "fare")

In [144]:
data

,pclass,age,embarked,fare
0,3,22.0,S,7.2500
2,3,26.0,S,7.9250
3,1,35.0,S,53.1000
4,3,35.0,S,8.0500
5,3,NaN,Q,8.4583
...,...,...,...,...
886,2,27.0,S,13.0000
887,1,19.0,S,30.0000
888,3,NaN,S,23.4500
889,1,26.0,C,30.0000


In [145]:
data.isnull().sum()

pclass        0
age         162
embarked      0
fare          0
dtype: int64

In [146]:
data.shape

(775, 4)

In [147]:
data.isnull().sum()

pclass        0
age         162
embarked      0
fare          0
dtype: int64

In [148]:
x = data.drop("fare", axis="columns")
y = data["fare"]

In [149]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Завдання 2
Створіть Pipeline для обробки даних

In [150]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder


num_col = x.select_dtypes(include="number").columns
cat_col = x.select_dtypes(include="object").columns

num_transformer = Pipeline(steps=[("imputing", SimpleImputer(strategy="mean"))])

cat_transformer = Pipeline(steps=[("imputing", SimpleImputer(strategy="most_frequent")),
                                  ("encoding", OrdinalEncoder())])


preprocessor = ColumnTransformer(transformers=[("numericals", num_transformer, num_col),
                                               ("categorical", cat_transformer, cat_col)])

preprocessor

ColumnTransformer(transformers=[('numericals',
                                 Pipeline(steps=[('imputing',
                                                  SimpleImputer())]),
                                 Index(['pclass', 'age'], dtype='object')),
                                ('categorical',
                                 Pipeline(steps=[('imputing',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoding',
                                                  OrdinalEncoder())]),
                                 Index(['embarked'], dtype='object'))])

# Завдання 3
Створіть остаточну модель та натренуйте її

In [151]:
from sklearn.tree import DecisionTreeRegressor

model = Pipeline(steps=[("preprocessor", preprocessor),
                        ("model", DecisionTreeRegressor())])

In [152]:
model.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numericals',
                                                  Pipeline(steps=[('imputing',
                                                                   SimpleImputer())]),
                                                  Index(['pclass', 'age'], dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputing',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoding',
                                                                   OrdinalEncoder())]),
                                                  Index(['embarked'], dtype='object'))])),
                ('model', DecisionTreeRegressor())])

# Завдання 4
Виведіть основні метрики моделі

In [153]:
from sklearn import metrics

y_pred = model.predict(x_test)

metrics_df = pd.DataFrame()

def get_metrics(y_pred, y_test, name="model"):
  metrics_df.loc["MAE", name] = metrics.mean_absolute_error(y_test, y_pred)
  metrics_df.loc["MSE", name] = metrics.mean_squared_error(y_test, y_pred) ** 0.5
  metrics_df.loc["R2", name] = metrics.r2_score(y_test, y_pred)

  return metrics_df.round(2)

fin_merics = pd.DataFrame()


fin_merics["train"] = get_metrics(model.predict(x_train), y_train)
fin_merics["test"] = get_metrics(model.predict(x_test), y_test)

fin_merics

,train,test
MAE,4.29,9.47
MSE,7.09,14.83
R2,0.70,0.13


# Завдання 5
Змініть параметри моделі та попробуйте покращити результат

Попробуйте:
* не видаляти викиди
* змінити обробку даних(imputer та кодування категоріальних даних)
* змінити параметри дерева(глибина, кількість точок у листках тощо

In [154]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module2/data/ship_passengers.csv",
                 index_col="Unnamed: 0")

df = df[['pclass', 'age', 'embarked', 'fare']]
df.head()


x = data.drop("fare", axis="columns")
y = data["fare"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [155]:
model = Pipeline(steps=[("preprocessor", preprocessor),
                        ("model", DecisionTreeRegressor(max_depth=4))])

In [156]:
model.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numericals',
                                                  Pipeline(steps=[('imputing',
                                                                   SimpleImputer())]),
                                                  Index(['pclass', 'age'], dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputing',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoding',
                                                                   OrdinalEncoder())]),
                                                  Index(['embarked'], dtype='object'))])),
                ('model', DecisionTreeRegressor(max_depth=4))])

In [157]:
y_pred = model.predict(x_test)

metrics_df = pd.DataFrame()

def get_metrics(y_pred, y_test, name="model"):
  metrics_df.loc["MAE", name] = metrics.mean_absolute_error(y_test, y_pred)
  metrics_df.loc["MSE", name] = metrics.mean_squared_error(y_test, y_pred) ** 0.5
  metrics_df.loc["R2", name] = metrics.r2_score(y_test, y_pred)

  return metrics_df.round(2)

fin_merics = pd.DataFrame()


fin_merics["train"] = get_metrics(model.predict(x_train), y_train)
fin_merics["test"] = get_metrics(model.predict(x_test), y_test)

fin_merics

,train,test
MAE,6.27,8.57
MSE,8.77,13.27
R2,0.53,0.30


# Завдання 6
Збережіть модель

In [159]:
import joblib

joblib.dump(model, "model.pkl")

['model.pkl']